#  Simulationen mit Python

- Praktische Umgebung mit Jupyter
- Kann komplexes Interface einfach implementieren
- Ganze Macht von Python steht zur Verfügung
- Simpy ist eine gute Bibliothek für Simulationen
- Viele Möglichkeiten der Visualisierung
- Modernes Framework
- Viele Ressourcen im Internet

# Beispiel Self-Checkout

Simuliere die Situation in einem Einkaufsladen mit Self-Checkout

In [9]:
import simpy
import random

class Store():
    def __init__(self, env, num_cashiers, num_checkouts):
        self.env = env
        self.cashier = simpy.Resource(env, num_cashiers)
        self.checkout = simpy.Resource(env, num_checkouts)
        
    def pay_at_checkout(self, customer):
        if customer.age < 50:
            time_needed = random.randint(1, 4)
        elif customer.age < 60:
            time_needed = random.randint(1, 6)
        else:
            time_needed = random.randint(10, 20)
        yield self.env.timeout(time_needed)
        
    def pay_at_cashier(self, customer):
        yield self.env.timeout(random.randint(1, 2))

In [2]:
def go_shopping(env, customer, store):
    arrival_time = env.now
    G['customers_waiting'] += 1
    
    if G['customers_waiting'] > 10:
        store.cashier = simpy.Resource(env, G['max_num_cashiers'])
    else:
        store.cashier = simpy.Resource(env, G['min_num_cashiers'])
    
    with store.cashier.request() as request:
        yield request
        yield env.process(store.pay_at_cashier(customer))
        
    with store.checkout.request() as request:
        yield request
        yield env.process(store.pay_at_checkout(customer))
    
    G['wait_times'].append(env.now - arrival_time)
    G['customers_waiting'] -= 1

In [3]:
class Customer():
    def __init__(self, id):
        self.id = id
        self.age = random.randint(20, 70)

In [4]:
def run_store(env, num_cashiers, num_checkouts):
    store = Store(env, num_cashiers, num_checkouts)
    
    for i in range(20):
        customer = Customer(i)
        env.process(go_shopping(env, customer, store))
        
    i = 0
    while True:
        yield env.timeout(random.randint(1, 3))
        
        i += 1
        customer = Customer(i)
        env.process(go_shopping(env, customer, store))

In [5]:
def run_simulation():
    random.seed(42)
    num_cashiers = G['init_num_cashiers']
    num_checkouts = G['init_num_checkouts']
    
    env = simpy.Environment()
    env.process(run_store(env, num_cashiers, num_checkouts))
    env.run(until=90)
    
    return env

In [6]:
# Globals

G = {
    'wait_times': [],
    'customers_waiting': 0,
    'max_num_cashiers': 10,
    'min_num_cashiers': 2,
    'init_num_cashiers': 2,
    'init_num_checkouts': 4,
}

In [12]:
import statistics

# Change testing parameter
G['init_num_checkouts'] = 8

simulation = run_simulation()
print(f"{statistics.mean(G['wait_times'])}")

11.307692307692308


# Jupyter

- Code kann in Zellen separiert werden
- Kann nur einzellne Zellen ausführen
- Kann auch Skripte importiren

# Simpy

- Framework für Simulationen
- Nimmt viel der Arbeit ab
- Gute Dokumentation
- Kann Simulationen am Stück rechnen oder in Echtzeit
- Keine direkte Visualisierung